In [1]:
!pip -q install timm

In [2]:
!pip -q install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.3/838.3 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 892.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
data_dir = '/content/thedataset/'

In [7]:
import os
from PIL import Image

# Path to the directory where your images are stored
image_dir = '/content/drive/MyDrive/CampusVisionChallengeFinal.zip'

# FOLDER

In [ ]:
import zipfile
import os
from PIL import Image

# Path to your .zip file (adjust if it's stored in Google Drive or a different location)
zip_file_path = '/content/drive/MyDrive/CampusVisionChallengeFinal.zip'
unzip_dir = '/content/thedataset/'  # Directory where the images will be extracted

# Step 1: Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)
    print(f"Dataset unzipped to {unzip_dir}")

Dataset unzipped to /content/thedataset/


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths
dataset_dir = '/content/thedataset/MergeDataset'  # Path to the main dataset folder
output_dir = '/content/thedataset/'  # Path to save the train/val splits

# Split ratio
val_split = 0.2  # 20% for validation

# Create output directories
os.makedirs(output_dir, exist_ok=True)
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_dir, split), exist_ok=True)

# Iterate over the 10 class folders
for folder_name in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder_name)

    if os.path.isdir(folder_path):
        # Create train and val directories for each class folder
        for split in ['train', 'val']:
            os.makedirs(os.path.join(output_dir, split, folder_name), exist_ok=True)

        # List all image files in the folder
        images = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

        # Split the dataset into train and val
        train_images, val_images = train_test_split(images, test_size=val_split, random_state=42)

        # Copy the images to the respective directories
        for img in train_images:
            shutil.copy(os.path.join(folder_path, img), os.path.join(output_dir, 'train', folder_name, img))
        for img in val_images:
            shutil.copy(os.path.join(folder_path, img), os.path.join(output_dir, 'val', folder_name, img))

        print(f"Completed splitting for folder: {folder_name}")

print("Dataset split completed successfully!")


Completed splitting for folder: Simrall Hall
Completed splitting for folder: Walker Hall
Completed splitting for folder: Lee Hall
Completed splitting for folder: Old Main
Completed splitting for folder: Carpenter Hall
Completed splitting for folder: Butler Hall
Completed splitting for folder: Swalm Hall
Completed splitting for folder: McCool Hall
Completed splitting for folder: McCain Hall
Completed splitting for folder: Student Union
Dataset split completed successfully!


# working well. Using resnet34.without no tunning. hope to ward to do better

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, log_loss
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import os
import matplotlib.pyplot as plt

In [9]:
from PIL import Image, ImageFilter
import random

In [10]:
class GaussianBlur(object):
    """Apply Gaussian blur with random sigma"""
    def __init__(self, radius_min=0.1, radius_max=2.0):
        self.radius_min = radius_min
        self.radius_max = radius_max

    def __call__(self, img):
        radius = random.uniform(self.radius_min, self.radius_max)
        return img.filter(ImageFilter.GaussianBlur(radius=radius))

def get_data_transforms():
    """
    Define data transformations for training and validation.
    Ensures consistent image sizes throughout the pipeline.
    """
    data_transforms = {
        'train': transforms.Compose([
            # First resize to a slightly larger size
            transforms.Resize(580),
            # Then always crop to target size
            transforms.RandomCrop(512),

            # Color augmentations
            transforms.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1
            ),

            # Geometric transformations
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomApply([
                transforms.RandomRotation(10)
            ], p=0.3),

            # Weather and lighting simulations
            transforms.RandomApply([GaussianBlur(0.1, 2.0)], p=0.2),

            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),

            # Random erasing (after normalization)
            transforms.RandomErasing(
                p=0.2,
                scale=(0.02, 0.1),
                ratio=(0.3, 3.3),
                value='random'
            )
        ]),

        'val': transforms.Compose([
            transforms.Resize(580),
            transforms.CenterCrop(512),  # Consistent crop size
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])
    }
    return data_transforms

In [11]:
class RobustImageFolder(datasets.ImageFolder):
    """Custom ImageFolder that handles corrupt images"""
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (OSError, IOError) as e:
            print(f"Warning: Skipping corrupt image at index {index}")
            valid_idx = (index + 1) % len(self)
            return super().__getitem__(valid_idx)

def load_data(data_dir, batch_size=32):
    """
    Load and prepare data loaders
    Reduced batch size and number of workers for better stability
    """
    data_transforms = get_data_transforms()
    image_datasets = {x: RobustImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x])
                     for x in ['train', 'val']}

    dataloaders = {x: DataLoader(image_datasets[x],
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=2,  # Reduced number of workers
                                pin_memory=True)  # Added pin_memory for better performance
                  for x in ['train', 'val']}

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    return dataloaders, dataset_sizes, class_names

In [12]:
# Step 3: Define the Model
# ------------------------
def create_model(num_classes):
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

    # Get the input dimensions of the last layer (fully connected layer)
    num_ftrs = model.fc.in_features

    # Replace the last layer with a new one that has `num_classes` outputs
    model.fc = nn.Linear(num_ftrs, num_classes)

    return model

In [13]:

# Step 4: Train the Model
# -----------------------
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    best_model_wts = model.state_dict()  # Store the best model weights
    best_acc = 0.0  # Track the best validation accuracy

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set the model to training mode
            else:
                model.eval()   # Set the model to evaluation mode

            running_loss = 0.0  # Track the loss
            running_corrects = 0  # Track the number of correct predictions

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)  # Move data to the GPU/CPU

                optimizer.zero_grad()  # Clear the gradients

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)  # Forward pass
                    _, preds = torch.max(outputs, 1)  # Get the class with the highest score
                    loss = criterion(outputs, labels)  # Calculate the loss

                    if phase == 'train':
                        loss.backward()  # Backpropagate the loss
                        optimizer.step()  # Update the weights

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                torch.save(model.state_dict(), "best_model.pth")
                print(f"Saved new best model")

    print(f'Best val Acc: {best_acc:.4f}')

    model.load_state_dict(best_model_wts)
    return model

In [14]:

# Step 5: Evaluate the Model
# --------------------------
def evaluate_model(model, dataloader, criterion, class_names):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    all_probs = []
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss

            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)  # Get the class with the highest score
            probs = torch.nn.functional.softmax(outputs, dim=1)  # Get probabilities

            all_labels.extend(labels.cpu().numpy())  # Store true labels
            all_preds.extend(preds.cpu().numpy())    # Store predicted labels
            all_probs.extend(probs.cpu().numpy())    # Store probabilities

    avg_loss = running_loss / len(dataloader.dataset)  # Calculate average loss

    # Calculate overall metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    logloss = log_loss(all_labels, all_probs)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'Log Loss: {logloss:.4f}')

    return accuracy, precision, recall, f1, logloss

In [15]:
dataloaders, dataset_sizes, class_names = load_data(data_dir)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = create_model(num_classes=len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=20)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 168MB/s]


Epoch 0/19
----------
train Loss: 1.7336 Acc: 0.5850
val Loss: 0.7313 Acc: 0.8879
Saved new best model
Epoch 1/19
----------
train Loss: 0.5375 Acc: 0.8888
val Loss: 0.2052 Acc: 0.9576
Saved new best model
Epoch 2/19
----------
train Loss: 0.2300 Acc: 0.9517
val Loss: 0.1039 Acc: 0.9778
Saved new best model
Epoch 3/19
----------
train Loss: 0.1321 Acc: 0.9728
val Loss: 0.0715 Acc: 0.9857
Saved new best model
Epoch 4/19
----------
train Loss: 0.0934 Acc: 0.9807
val Loss: 0.0551 Acc: 0.9893
Saved new best model
Epoch 5/19
----------
train Loss: 0.0701 Acc: 0.9848
val Loss: 0.0457 Acc: 0.9913
Saved new best model
Epoch 6/19
----------
train Loss: 0.0597 Acc: 0.9875
val Loss: 0.0400 Acc: 0.9925
Saved new best model
Epoch 7/19
----------
train Loss: 0.0478 Acc: 0.9889
val Loss: 0.0380 Acc: 0.9913
Epoch 8/19
----------
train Loss: 0.0407 Acc: 0.9908
val Loss: 0.0326 Acc: 0.9937
Saved new best model
Epoch 9/19
----------
train Loss: 0.0345 Acc: 0.9923
val Loss: 0.0307 Acc: 0.9941
Saved new be

KeyboardInterrupt: 

In [16]:
# Evaluate the model on the validation set
print("Validation Set Performance:")
evaluate_model(model, dataloaders['val'], criterion, class_names)

# Save the best model
torch.save(model.state_dict(), "best_model.pth")

Validation Set Performance:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


Accuracy: 0.9945
Precision: 0.9945
Recall: 0.9945
F1 Score: 0.9945
Log Loss: 0.0300


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Load the image
def load_image(image_path):
    # Open the image file
    image = Image.open(image_path).convert('RGB')
    return image

# Define a transformation pipeline for resizing and normalization
def preprocess_image(image, target_size=(256, 256)):
    transform = transforms.Compose([
        transforms.Resize(target_size),  # Resize to 256x256
        transforms.ToTensor(),           # Convert to a PyTorch tensor
        transforms.Normalize(mean=[0.4608, 0.4610, 0.4559], std=[0.2312, 0.2275, 0.2638])  # Normalize with same mean and std in which it was trained
    ])
    return transform(image)

# Perform inference with resizing and normalization
def infer_with_resizing(model, image_path, class_names):
    # Load and preprocess the image
    image = load_image(image_path)
    image = preprocess_image(image)

    # Add a batch dimension (1, C, H, W)
    image = image.unsqueeze(0)

    # Move the input tensor to the same device as the model
    image = image.to(device)

    # Make sure the model is in evaluation mode
    model.eval()

    # Perform inference
    with torch.no_grad():
        output = model(image)

    # Get the predicted class
    _, predicted = torch.max(output, 1)
    predicted_class = class_names[predicted.item()]

    return predicted_class

In [ ]:
# Load the model and class names
model = create_model(num_classes=len(class_names))
model.load_state_dict(torch.load("best_model.pth"))
model = model.to(device)

# Perform inference
image_path = '/content/thedataset/val/Carpenter Hall/20241013_220952549_iOS.png'
predicted_class = infer_with_resizing(model, image_path, class_names)
print(f"Predicted class: {predicted_class}")

<ipython-input-23-493d6d341e1c>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Predicted class: Carpenter Hall


In [ ]:
# Perform inference
image_path = '/content/Screenshot (992).png'
predicted_class = infer_with_resizing(model, image_path, class_names)
print(f"Predicted class: {predicted_class}")

Predicted class: Walker Hall
